<div style="text-align: center;" class="alert alert-block alert-info"> 
    <h1 style="font-weight: bold; margin: 0;">Aircraft Anomaly Detection - Main</h1>
</div>


### markdown bien fait pour expliquer 

In [5]:
import sys
import os
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import numpy as np
PROJECT_ROOT = os.path.abspath(os.path.join(".."))
if PROJECT_ROOT not in sys.path:
    sys.path.insert(0, PROJECT_ROOT)

from aircraft_project.data import load_data, save_data
from aircraft_project.feature import split_columns, create_time_index, normalisation, standard_deviation
from aircraft_project.anomaly_detection import detect_abnormal_windows, get_abnormal_windows
from aircraft_project.config import get_filepath

### Data loading

In [6]:
df_aircraft = load_data("dataset.csv")
df_aircraft.head()

,day_cycle_window,p1,p2,p3,p4,p5,p6,p7,p8,p9,p10,p11
0,1_1_1,-0.805191,-1.141993,-1.328327,-1.516693,-1.373962,-0.80827,-1.070547,-0.504352,2.807146,0.495528,-1.246737
1,1_1_1,-0.805191,-1.134902,-1.320393,-1.505848,-1.361560,-0.80784,-1.064212,-0.516000,2.894177,0.494467,-1.244472
2,1_1_1,-0.805191,-1.117304,-1.307167,-1.505848,-1.361560,-0.80784,-1.064212,-0.516000,2.981904,0.429370,-1.244472
3,1_1_1,-0.814231,-1.114206,-1.287593,-1.442302,-1.287565,-0.80784,-1.045557,-0.516000,3.063298,0.348056,-1.244472
4,1_1_1,-0.848104,-1.112276,-1.285393,-1.429076,-1.270722,-0.80784,-1.017575,-0.664906,3.049237,0.118828,-1.270237


explications +

In [10]:
df_aircraft.isna().sum().sum()

np.int64(0)

explication en anglais new columns

In [11]:
df_aircraft = split_columns(df_aircraft)
df_aircraft = create_time_index(df_aircraft)
df_aircraft.head()

,day_cycle_window,p1,p2,p3,p4,p5,p6,p7,p8,p9,p10,p11,day,cycle,window,time_index
0,1_1_1,-0.805191,-1.141993,-1.328327,-1.516693,-1.373962,-0.80827,-1.070547,-0.504352,2.807146,0.495528,-1.246737,1,1,1,1
1,1_1_1,-0.805191,-1.134902,-1.320393,-1.505848,-1.361560,-0.80784,-1.064212,-0.516000,2.894177,0.494467,-1.244472,1,1,1,2
2,1_1_1,-0.805191,-1.117304,-1.307167,-1.505848,-1.361560,-0.80784,-1.064212,-0.516000,2.981904,0.429370,-1.244472,1,1,1,3
3,1_1_1,-0.814231,-1.114206,-1.287593,-1.442302,-1.287565,-0.80784,-1.045557,-0.516000,3.063298,0.348056,-1.244472,1,1,1,4
4,1_1_1,-0.848104,-1.112276,-1.285393,-1.429076,-1.270722,-0.80784,-1.017575,-0.664906,3.049237,0.118828,-1.270237,1,1,1,5


Normalisation

In [13]:
df_normalized = normalisation(df_aircraft, ["p1", "p2", "p3", "p4", "p5", "p6", "p7", "p8", "p9", "p10", "p11"])
df_normalized.describe()

,p1,p2,p3,p4,p5,p6,p7,p8,p9,p10,p11,day,cycle,window,time_index
count,162702.000000,162702.000000,162702.000000,162702.000000,162702.000000,162702.000000,162702.000000,162702.000000,162702.000000,162702.000000,162702.000000,162702.000000,162702.000000,162702.000000,162702.000000
mean,0.390474,0.351630,0.277635,0.287819,0.338086,0.426915,0.510615,0.430292,0.625542,0.682728,0.256992,2.586323,2.337716,73.277587,81351.500000
std,0.121561,0.103553,0.083479,0.085186,0.103030,0.101635,0.130987,0.122064,0.045543,0.056665,0.080711,1.115984,1.170402,50.556205,46968.166086
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000
25%,0.307952,0.301638,0.225112,0.245939,0.274993,0.370095,0.431215,0.340400,0.598471,0.654472,0.203830,2.000000,1.000000,30.000000,40676.250000
50%,0.395564,0.370307,0.281092,0.300376,0.350144,0.442565,0.534985,0.428784,0.623951,0.668123,0.255743,3.000000,2.000000,63.000000,81351.500000
75%,0.480916,0.419660,0.332668,0.342077,0.411486,0.496576,0.604699,0.528240,0.648741,0.703120,0.308389,4.000000,3.000000,114.000000,122026.750000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,4.000000,5.000000,181.000000,162702.000000


detection d'anomalies

In [16]:
df_std = standard_deviation(df_normalized)
detect_abnormal_windows(df_std, n_contamination=0.01)

df_std[df_std["anomaly"] == -1][['day_cycle_window', 'anomaly']].head()

/Users/joshuaverhaeghe/aircraft-anomaly-detection/aircraft_project/feature.py:36: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_ecart_type = pd.concat([df_ecart_type, pd.DataFrame([[w] + ecart_type.tolist()], columns=df_ecart_type.columns)], ignore_index=True)


,day_cycle_window,anomaly
61,1_1_62,-1
67,1_1_68,-1
68,1_2_1,-1
69,1_2_2,-1
367,2_1_2,-1


blabla sur choix parametre et maniere de verifier le resultat

# Aircraft Anomaly Detection

Ce notebook illustre un projet complet de détection d’anomalies sur des données d’avion.  
Il inclut :
- Chargement et exploration des données
- Pré-traitement et feature engineering
- Normalisation
- Détection d’anomalies
- Visualisation des résultats


<h2 style="color:white;">2️⃣ Exploration des données</h2>


# Sequence to Sequence Learning with Neural Networks

<a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/"><img alt="Creative Commons License" align="left" src="https://i.creativecommons.org/l/by-nc-sa/4.0/80x15.png" /></a>&nbsp; Dennis G. Wilson | Based on Ben Trevett's [tutorials](https://github.com/bentrevett/pytorch-seq2seq).

<div class="alert alert-block alert-warning">
    
First and before anything, you should rename your notebook file with 'student' inside.
</div>

<div class="alert alert-block alert-success">
    
<h1> <b> Conclusion : </b></h1>

bla bla bla
 <b>en gras </b> 
</div>

<div class="alert alert-block alert-info">
    
<b> Exercise 1.1.1: load dataset</b>

- Use the method 'read_csv()' from the pandas library to load the data.
- Make sure to specify '"Id"' for the 'index_col' parameter.

You can find the <a href= https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html> documention right here</a>. If you want example of utilization, search "pandas read_csv example" on internet.

</div>